In [50]:
import os
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize

In [8]:
data_path = os.path.join(os.getcwd(), "final_rawcounts_merged.csv")
print(data_path)

/Users/gautam/Downloads/KU_Leuven_Bioinformatics_Masters/datathon_ambulance/final_rawcounts_merged.csv


In [12]:
data = pd.read_csv(data_path)
X = data.iloc[:, [2, 3, 5, 6, 7]]
y = data.iloc[:, 4]

In [21]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True)

In [22]:
scaler = StandardScaler()
scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [23]:
model = Sequential()
model.add(Dense(50, input_shape = (5,), kernel_initializer = 'normal', activation = 'relu'))
model.add(Dropout(rate = 0.4))
model.add(Dense(25, activation = 'relu'))
model.add(Dropout(rate = 0.2))
model.add(Dense(1, kernel_initializer='normal'))
# Compile model
model.compile(loss = 'mse', optimizer = 'rmsprop', metrics = 'mae')

In [24]:
model.fit(x = X_train, y = y_train, validation_split = 0.2, epochs=1000, batch_size=32)

Epoch 1/1000
Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x137a15da0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x137a15da0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original erro

In [25]:
model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 9ms/step - loss: 17.8427 - mae: 2.5558


[17.84273910522461, 2.5558180809020996]

In [26]:
pred = model.predict(X_test)

Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x1379aaca0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x1379aaca0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: co

In [40]:
np.mean((y_test.to_numpy(dtype = 'float32') - np.ndarray.flatten(pred))**2)

17.842731

In [38]:
y_test.to_numpy(dtype = 'float32')

array([  0.,   4., 171.,   0.,  83.,  30.,   0., 105.,  35., 250.,  28.,
         0.,   0.,   0.,  34.,  79.,  12.,   0.,  54.,   0.,  33.,  50.,
         0.,  40.,  53.,   1.,   3.,   0.,   6.,  45.,  34.,   9.,   0.,
         0.,  44.,  48.,   0., 170.,  76.,   0.,   0., 336.,  20.,  97.,
       142.,   7.,  22.,   0.,  99.,   0., 128., 209.,  12.,   0.,   0.,
         0.,  13.,  65.,   0.,   0.,  38.,  14.,   1.,  41.,  75.],
      dtype=float32)

In [57]:
data_without_aed_counts = X.drop(columns = 'aed_count')
x0 = data['aed_count'].to_numpy()
total_aeds = 2*np.sum(x0)

def objective(x):
    frame = data_without_aed_counts
    frame['aed_count'] = pd.Series(x)
    res = np.ndarray.flatten(model.predict(frame))
    res[res<0] = 0
    res_norm = np.linalg.norm(res)
    return res_norm

def constraint1(x):
    return np.sum(x) - total_aeds

cons = {'type' : 'eq', 'fun' : constraint1}

res = minimize(objective, x0, constraints = cons, method = 'SLSQP')

11/11 [==============================] - 0s 3ms/step


In [58]:
table = pd.DataFrame(data['postal_code'])
table['aed_count'] = res.x
table.to_csv('randomforest_aed_pred_nn.csv', sep=',')

In [52]:
clf = RandomForestRegressor(n_estimators = 1000)
clf.fit(X_train_raw.to_numpy(), y_train.to_numpy())
pred_clf = clf.predict(X_test_raw)

/Users/gautam/miniconda3/envs/pytorch_env/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [54]:
np.mean((((pred_clf - np.ndarray.flatten(pred)) / 2) - y_test)**2)

TypeError: only integer scalar arrays can be converted to a scalar index